<a href="https://colab.research.google.com/github/ThePain99/PrediccionAcademica/blob/master/CodeFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from google.colab import files
uploaded = files.upload()

df1 = pd.read_excel('dataset.xlsx', sheet_name='Primero')
df2 = pd.read_excel('dataset.xlsx', sheet_name='Segundo')

nombres_cursos = df1.columns.tolist()


##Visualizacion de Datos

In [ ]:
df1

In [ ]:
df2

##Preprocesamiento de datos

In [ ]:
df1.isnull()

In [ ]:
df2.isnull()

In [ ]:
df1.isnull().sum()

In [ ]:
df2.isnull().sum()

In [ ]:
df1.fillna(0, inplace=True)
df1 = df1.apply(pd.to_numeric, errors='coerce')
df1.fillna(0, inplace=True)

In [ ]:
df2.fillna(0, inplace=True)
df2 = df2.apply(pd.to_numeric, errors='coerce')
df2.fillna(0, inplace=True)

##Construcción del Modelo

In [ ]:
x = df1[nombres_cursos].values
y = df2[nombres_cursos].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=(10,)),  # Capa de entrada con 9 neuronas de entrada (notas cursos anteriores)
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)  # Capa de salida con 9 neuronas (notas cursos siguientes)
])

In [ ]:
model.compile(optimizer='adam', loss='mse')

##Entrenamiento del Modelo

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=4, verbose=0, callbacks=[early_stopping])

In [ ]:
loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Pérdida en el conjunto de prueba: {loss}")

##Prueba de Predicción

In [ ]:
uploaded = files.upload()
test_data = pd.read_excel('notas_test.xlsx')

caracteristicas_prediccion = test_data.values

predicciones = model.predict(caracteristicas_prediccion)
df_predicciones = pd.DataFrame(predicciones, columns=nombres_cursos)
print("Predicciones:")
print(df_predicciones)

In [ ]:
def clasificar_notas(nota):
    if nota <= 10:
        return 'Necesita mejorar significativamente'
    elif nota <= 14:
        return 'Necesita mejorar'
    elif nota <= 18:
        return 'Tiene un buen rendimiento'
    else:
        return 'Tiene un excelente rendimiento'

df_predicciones_categorias = df_predicciones.applymap(clasificar_notas)

print("Predicciones:")
print(df_predicciones_categorias)

In [ ]:
categorias = df_predicciones_categorias.stack().value_counts()

fig, ax = plt.subplots()
ax.pie(categorias, labels=categorias.index, autopct='%1.1f%%', startangle=90)

ax.axis('equal')
plt.title('Categorías de Notas')
plt.show()

In [ ]:
limites_categorias = [0, 10, 14, 18, 20]
etiquetas_categorias = ['Necesita apoyo urgente', 'Necesita mejorar', 'Necesita mejorar detalles', 'Tiene buen rendimiento']
datos_categorias_por_curso = []

for curso in df_predicciones.columns:
    notas = df_predicciones[curso]
    categorias = pd.cut(notas, bins=limites_categorias, labels=etiquetas_categorias)
    datos_categorias = categorias.value_counts()
    datos_categorias_por_curso.append(datos_categorias)

datos_categorias_por_curso_filtrados = [datos[datos > 0] for datos in datos_categorias_por_curso]

if len(datos_categorias_por_curso_filtrados) == 0:
    print("No hay datos de categorías válidos para mostrar.")
else:
    fig, axes = plt.subplots(nrows=len(datos_categorias_por_curso_filtrados), figsize=(8, 6*len(datos_categorias_por_curso_filtrados)))

    for i, (curso, datos_categorias) in enumerate(zip(df_predicciones.columns, datos_categorias_por_curso_filtrados)):
        ax = axes[i]
        ax.pie(datos_categorias, labels=datos_categorias.index, autopct='%1.1f%%', startangle=90)
        ax.set_title(curso)

    plt.tight_layout()
    plt.show()

##Exportar Resultados

In [ ]:
import xlsxwriter

output_file = "resultados.xlsx"

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    df_predicciones.to_excel(writer, sheet_name="Predicciones", index=False)
    df_predicciones_categorias.to_excel(writer, sheet_name="Mejoras", index=False)

    workbook = writer.book
    worksheet_predicciones = writer.sheets["Predicciones"]

    for i, col in enumerate(df_predicciones.columns):
        column_width = max(df_predicciones[col].astype(str).map(len).max(), len(col)) + 2
        worksheet_predicciones.set_column(i, i, column_width)

    worksheet_predicciones_cat = writer.sheets["Mejoras"]
    for i, col in enumerate(df_predicciones_categorias.columns):
        column_width = max(df_predicciones_categorias[col].astype(str).map(len).max(), len(col)) + 2
        worksheet_predicciones_cat.set_column(i, i, column_width)


files.download(output_file)